# Pre-Processing

In [2]:
from data import test_data, train_data

vocab = list(set([w for text in train_data.keys() for w in text.split(' ')]))
vocab_size = len(vocab)
print(f'{vocab_size} unique words found')

word_to_idx = { w: i for i, w in enumerate(vocab)}
idx_to_word = { i: w for i, w in enumerate(vocab)}
print(word_to_idx['sad'])
print(idx_to_word[0])

18 unique words found
2
i


In [3]:
import numpy as np

def createInputs(text:str):
    inputs = []
    for w in text.split(' '):
        v = np.zeros((vocab_size, 1))
        v[word_to_idx[w]] = 1
        inputs.append(v)
    return inputs


# RNN Setup class

In [4]:
import numpy as np
from numpy.random import randn

class RNN:
    def __init__(self, input_size, output_size, hidden_size=64):
        # Weights
        
        self.Whh = randn(hidden_size, hidden_size) / 1000
        self.Wxh = randn(hidden_size, input_size) / 1000
        self.Why = randn(output_size, hidden_size) / 1000
        
        # Biases
        self.bh = np.zeros((hidden_size, 1))
        self.by = np.zeros((output_size, 1))
        
    def forward(self, inputs):
        h = np.zeros((self.Whh.shape[0], 1))
        
        self.last_inputs = inputs
        self.last_hs = { 0: h }
        for i, x in enumerate(inputs):
           h = np.tanh(self.Wxh @ x + self.Whh @ h + self.bh)
        y = self.Why @ h + self.by
        
        return y, h
    
    def backprop(self, d_y, learn_rate=2e-2):
        pass

    
def softmax(xs):
    return np.exp(xs) / sum(np.exp(xs))
    
rnn = RNN(vocab_size, 2)
inputs = createInputs('i am very good')
out, h = rnn.forward(inputs)
probs = softmax(out)
print(probs)

[[0.50000237]
 [0.49999763]]


# Setup

In [5]:
for x, y in train_data.items():
    inputs = createInputs(x)
    target = int(y)
    
    # forward 
    out, _ = rnn.forward(inputs)
    prob = softmax(out)
    
    d_L_d_y = probs
    d_L_d_y[target] -= 1
    
    # backward
    rnn.backprop(d_L_d_y)